In [ ]:
top_left = [51.710756, -0.516730]
bottom_left = [51.293150, -0.516387]
bottom_right = [51.285499, 0.227786]
top_right = [51.687818, 0.269924]

In [1]:
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile, mutual_info_regression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, OneHotEncoder, RobustScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [2]:
crime_features =                   ['Anti_social_behaviour_100',
                                    'Criminal_damage_and_arson_100',
                                    'Drugs_100',
                                    'Other_theft_100',
                                    'Public_order_100',
                                    'Vehicle_crime_100',
                                    'Violence_and_sexual_offences_100',
                                    'Theft_from_the_person_100',
                                    'Burglary_100',
                                    'Shoplifting_100',
                                    'Possession_of_weapons_100',
                                    'Other_crime_100',
                                    'Robbery_100',
                                    'Bicycle_theft_100',
                                    'Anti_social_behaviour_250',
                                    'Criminal_damage_and_arson_250',
                                    'Drugs_250',
                                    'Other_theft_250',
                                    'Public_order_250',
                                    'Vehicle_crime_250',
                                    'Violence_and_sexual_offences_250',
                                    'Theft_from_the_person_250',
                                    'Burglary_250',
                                    'Shoplifting_250',
                                    'Possession_of_weapons_250',
                                    'Other_crime_250',
                                    'Robbery_250',
                                    'Bicycle_theft_250',
                                    'Anti_social_behaviour_500',
                                    'Criminal_damage_and_arson_500',
                                    'Drugs_500',
                                    'Other_theft_500',
                                    'Public_order_500',
                                    'Vehicle_crime_500',
                                    'Violence_and_sexual_offences_500',
                                    'Theft_from_the_person_500',
                                    'Burglary_500',
                                    'Shoplifting_500',
                                    'Possession_of_weapons_500',
                                    'Other_crime_500',
                                    'Robbery_500',
                                    'Bicycle_theft_500',
                                    'Anti_social_behaviour_750',
                                    'Criminal_damage_and_arson_750',
                                    'Drugs_750',
                                    'Other_theft_750',
                                    'Public_order_750',
                                    'Vehicle_crime_750',
                                    'Violence_and_sexual_offences_750',
                                    'Theft_from_the_person_750',
                                    'Burglary_750',
                                    'Shoplifting_750',
                                    'Possession_of_weapons_750',
                                    'Other_crime_750',
                                    'Robbery_750',
                                    'Bicycle_theft_750',
                                    'Anti_social_behaviour_1000',
                                    'Criminal_damage_and_arson_1000',
                                    'Drugs_1000',
                                    'Other_theft_1000',
                                    'Public_order_1000',
                                    'Vehicle_crime_1000',
                                    'Violence_and_sexual_offences_1000',
                                    'Theft_from_the_person_1000',
                                    'Burglary_1000',
                                    'Shoplifting_1000',
                                    'Possession_of_weapons_1000',
                                    'Other_crime_1000',
                                    'Robbery_1000',
                                    'Bicycle_theft_1000',
                                    'Anti_social_behaviour_1250',
                                    'Criminal_damage_and_arson_1250',
                                    'Drugs_1250',
                                    'Other_theft_1250',
                                    'Public_order_1250',
                                    'Vehicle_crime_1250',
                                    'Violence_and_sexual_offences_1250',
                                    'Theft_from_the_person_1250',
                                    'Burglary_1250',
                                    'Shoplifting_1250',
                                    'Possession_of_weapons_1250',
                                    'Other_crime_1250',
                                    'Robbery_1250',
                                    'Bicycle_theft_1250',
                                    'Anti_social_behaviour_1500',
                                    'Criminal_damage_and_arson_1500',
                                    'Drugs_1500',
                                    'Other_theft_1500',
                                    'Public_order_1500',
                                    'Vehicle_crime_1500',
                                    'Violence_and_sexual_offences_1500',
                                    'Theft_from_the_person_1500',
                                    'Burglary_1500',
                                    'Shoplifting_1500',
                                    'Possession_of_weapons_1500',
                                    'Other_crime_1500',
                                    'Robbery_1500',
                                    'Bicycle_theft_1500']


In [ ]:
# Feature Selection

#Load Golden DF
#Change FilePath
local_golden_df = pd.read_csv("/home/willbanny/code/willbanny/Location-Analysis/raw_data/London_Data_Expanded_1.csv")

# Preprocess and select features
local_golden_df = local_golden_df.fillna(0)

dirty_city_features = local_golden_df.columns

city_features = []

for feature in dirty_city_features:
    if "250" in feature:
        city_features.append(feature)
    elif "500" in feature:
        city_features.append(feature)
    elif "750" in feature:
        city_features.append(feature)
    elif "100" in feature:
        city_features.append(feature)
    elif "1000" in feature:
        city_features.append(feature)
    elif "1250" in feature:
        city_features.append(feature)
    elif "1500" in feature:
        city_features.append(feature)
    elif "Score" in feature:
        if "1" not in feature:
            city_features.append(feature)


#Create a DataFrame which will be scaled

to_scale_df = pd.DataFrame()
for feature in city_features:
    to_scale_df[feature] = local_golden_df[feature]


#Scaling

#MinMax Scale the DataFrame

mm_scaler = MinMaxScaler()
mm_scaler.fit(to_scale_df)
mm_scaled_df = pd.DataFrame(mm_scaler.transform(to_scale_df), columns=city_features)

#Robust Scale the DataFrame

r_scaler = RobustScaler()
r_scaler.fit(to_scale_df)
r_scaled_df = pd.DataFrame(r_scaler.transform(to_scale_df), columns=city_features)


#Crimeless selection

crimeless_features = list(set(city_features) - set(crime_features))


mm_crimeless_non_PCA_df = pd.DataFrame()
r_crimeless_non_PCA_df = pd.DataFrame()

for feature in crimeless_features:

    r_crimeless_non_PCA_df[feature] = r_scaled_df[feature]
    mm_crimeless_non_PCA_df[feature] = mm_scaled_df[feature]


#PCA

#MinMax Crime PCA

mm_pca = PCA()
mm_pca.fit(mm_scaled_df)

mm_proj = mm_pca.transform(mm_scaled_df)
mm_proj = pd.DataFrame(mm_proj, columns=[f'PC{i}' for i in range(1, len(city_features) + 1)])

#MinMax Crimeless PCA

mm_crimeless_pca = PCA()
mm_crimeless_pca.fit(mm_crimeless_non_PCA_df)

mm_crimeless_proj = mm_crimeless_pca.transform(mm_crimeless_non_PCA_df)
mm_crimeless_proj = pd.DataFrame(mm_crimeless_proj, columns=[f'PC{i}' for i in range(1, len(crimeless_features) + 1)])

#Robust Crime PCA

r_pca = PCA()
r_pca.fit(r_scaled_df)

r_proj = r_pca.transform(r_scaled_df)
r_proj = pd.DataFrame(r_proj, columns=[f'PC{i}' for i in range(1, len(city_features )+ 1)])

#MinMax Crimeless PCA

r_crimeless_pca = PCA()
r_crimeless_pca.fit(r_crimeless_non_PCA_df)

r_crimeless_proj = r_crimeless_pca.transform(r_crimeless_non_PCA_df)
r_crimeless_proj = pd.DataFrame(r_crimeless_proj, columns=[f'PC{i}' for i in range(1, len(crimeless_features) + 1)])


#Care home selection

england_care_home_df = pd.read_csv('/home/willbanny/code/willbanny/Location-Analysis/raw_data/carehome_locations (3).csv')

#Only store those that fit the golden df dimensions

care_home_local_df = england_care_home_df[england_care_home_df["latitude"] <= local_golden_df["lat"].max()]
care_home_local_df = care_home_local_df[care_home_local_df["latitude"] >= local_golden_df["lat"].min() ]
care_home_local_df = care_home_local_df[care_home_local_df["longitude"] >= local_golden_df["lng"].min() ]
care_home_local_df = care_home_local_df[care_home_local_df["longitude"] <= local_golden_df["lng"].max() ]

#K-Means

#MinMax Crime K-means PCA

mm_crime_pca_km = KMeans(n_clusters= 3)
mm_crime_pca_km.fit(mm_proj)

#MinMax Crimeless K-means PCA

mm_crimeless_pca_km = KMeans(n_clusters= 3)
mm_crimeless_pca_km.fit(mm_crimeless_proj)

#Robust Crime K-means PCA

r_crime_pca_km = KMeans(n_clusters= 3)
r_crime_pca_km.fit(r_proj)

#Robust Crimeless K-means PCA

r_crimeless_pca_km = KMeans(n_clusters= 3)
r_crimeless_pca_km.fit(mm_crimeless_proj)




#MinMax Crime K-means Non_PCA

mm_crime_non_pca_km = KMeans(n_clusters= 3)
mm_crime_non_pca_km.fit(mm_scaled_df)

#MinMax Crimeless K-means Non_PCA

mm_crimeless_non_pca_km = KMeans(n_clusters= 3)
mm_crimeless_non_pca_km.fit(mm_crimeless_non_PCA_df)

#Robust Crime K-means Non_PCA

r_crime_non_pca_km = KMeans(n_clusters= 3)
r_crime_non_pca_km.fit(r_scaled_df)

#Robust Crimeless K-means Non_PCA

r_crimeless_non_pca_km = KMeans(n_clusters= 3)
r_crimeless_non_pca_km.fit(r_crimeless_non_PCA_df)


#Export df


export_df = local_golden_df[["lng","lat"]].copy()

export_df["MinMax_PCA_Crime_Labels"] = mm_crime_pca_km.labels_
export_df["Robust_PCA_Crime_Labels"] = r_crime_pca_km.labels_
export_df["MinMax_PCA_Crimeless_Labels"] = mm_crimeless_pca_km.labels_
export_df["Robust_PCA_Crimeless_Labels"] = r_crimeless_pca_km.labels_

export_df["MinMax_Non_PCA_Crime_Labels"] = mm_crime_non_pca_km.labels_
export_df["Robust__Non_PCA_Crime_Labels"] = r_crime_non_pca_km.labels_
export_df["MinMax_Non_PCA_Crimeless_Labels"] = mm_crimeless_non_pca_km.labels_
export_df["Robust__Non_PCA_Crimeless_Labels"] = r_crimeless_non_pca_km.labels_


#Change File_Path
# export_df.to_csv('/home/mih_sud/code/willbanny/Location-Analysis/raw_data/labels_export.csv')